In [1]:
# Import required modules
import sys
from pathlib import Path
import pandas as pd

In [2]:
project_root = str(Path.cwd().resolve().parent)
print(f"Project root: {project_root}")

Project root: C:\Users\Nitya Karthik A\ds4cg-job-analytics


In [3]:
# Add project root to sys.path for module imports
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.analysis import vram_usage

# Automatically reload modules before executing code
# This is useful for development to see changes without restarting the kernel.
%load_ext autoreload
# Reload all modules imported with %aimport every time before executing the Python code typed.
%autoreload 1
%aimport src.analysis.vram_usage, src.preprocess.preprocess, src.config.enum_constants

In [4]:
# Load the jobs DataFrame from DuckDB

efficiency_analysis = vram_usage.EfficiencyAnalysis(
	db_path='../data/slurm_data_small.db',
)

display(efficiency_analysis.jobs_df.head(10))
print(efficiency_analysis.jobs_df.shape)

Connected to ../data/slurm_data_small.db
Disconnected from ../data/slurm_data_small.db


,JobID,ArrayID,JobName,IsArray,Interactive,Account,User,Constraints,QOS,Status,...,GPUType,GPUMemUsage,GPUComputeUsage,CPUMemUsage,CPUComputeUsage,Queued,vram_constraint,allocated_vram,user_jobs,account_jobs
20,28982142,-1,p3,False,non-interactive,pi_allan_umass_edu,nazaninjafar_umass_edu,[],gypsum-phd-rtx8000,TIMEOUT,...,[rtx_8000],0.000000e+00,0.0,1.046647e+09,0.158025,0 days 00:00:01,<NA>,48,5,5
21,28989215,-1,inference_user_query_review_full.sh,False,non-interactive,pi_rrahimi_umass_edu,changzeng_umass_edu,['gpu:a100'],normal,COMPLETED,...,[a100],8.491159e+10,100.0,9.295657e+10,97.740585,0 days 00:04:43,320,640,5,16
22,29004768,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-A0A1P8AT95),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,OUT_OF_MEMORY,...,[l40s],9.625928e+08,100.0,3.691685e+10,16.136250,2 days 06:24:31,23,48,891,891
23,29004931,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-A0A1P8AUH4),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,OUT_OF_MEMORY,...,[l40s],4.865393e+08,100.0,3.864463e+10,16.197411,2 days 06:53:21,23,48,891,891
24,29004942,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-F4K5Y4),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,COMPLETED,...,[l40s],9.500099e+08,100.0,2.878319e+10,16.564669,2 days 07:05:05,23,48,891,891
26,29004957,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-Q9S9Q9),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,COMPLETED,...,[a40],9.059697e+08,100.0,2.126001e+10,13.485684,2 days 06:52:20,23,48,891,891
28,29004977,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-Q8GWC3),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,COMPLETED,...,[m40],1.389363e+08,100.0,9.670738e+09,16.328876,2 days 06:33:21,23,23,891,891
29,29005003,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-Q0WQN4),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,COMPLETED,...,[m40],2.502164e+08,100.0,9.522024e+09,15.754887,2 days 06:09:00,23,23,891,891
30,29005010,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-Q8GYL7),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,COMPLETED,...,[v100],6.878659e+08,100.0,2.109012e+10,16.048647,2 days 06:06:42,23,32,891,891
31,29005014,-1,nf-COLABFOLD_COLABFOLD_BATCH_(O64948-Q9LRA9),False,non-interactive,pi_bioch642_umass_edu,emccleary_umass_edu,['vram23'],normal,COMPLETED,...,[m40],2.513961e+08,100.0,1.088179e+10,16.628517,2 days 06:00:55,23,23,891,891


(9948, 30)


In [ ]:
# Load the jobs DataFrame from DuckDB

efficiency_analysis = vram_usage.EfficiencyAnalysis(
	db_path='../data/slurm_data_small.db',
)

display(efficiency_analysis.jobs_df.head(10))
print(efficiency_analysis.jobs_df.shape)

Connected to ../data/slurm_data.db


RuntimeError: Failed to load jobs DataFrame: Catalog Error: Table with name Jobs does not exist!
Did you mean "pg_constraint"?

LINE 1: SELECT * FROM Jobs
                      ^

In [ ]:
# Analyze workload efficiency of GPU users with no VRAM constraints who used 0 GB of VRAM
filtered_jobs = efficiency_analysis.filter_jobs_for_analysis(
    vram_constraint_filter=pd.NA,  # No VRAM constraints
    gpu_mem_usage_filter=0,  # Used 0 GB of VRAM
    allocated_vram_filter=pd.NA,  # No filter on allocated VRAM
    gpu_count_filter=pd.NA,  # No filter on GPU count
)

metrics_dict = efficiency_analysis.calculate_all_efficiency_metrics(filtered_jobs)


jobs_with_metrics = metrics_dict['jobs_with_efficiency_metrics']
users_with_metrics = metrics_dict['users_with_efficiency_metrics']
pi_accounts_with_metrics = metrics_dict['pi_accounts_with_efficiency_metrics']

# Set option to display all columns
pd.set_option('display.max_columns', None)
# Display the DataFrame
display(jobs_with_metrics.head(10))
# To revert to default settings (optional)
pd.reset_option('display.max_columns')
print(f"Jobs found: {len(jobs_with_metrics)}")

In [ ]:
# Run evaluate_cpu_gpu_balance
analysis_results = efficiency_analysis.evaluate_cpu_gpu_usage()
print("Efficiency summary stats:")

# Display key summary statistics
print("Total jobs:", analysis_results["total_jobs"])
print("Total GPU hours:", analysis_results["total_gpu_hours"])
print("Average VRAM efficiency:", f"{analysis_results['avg_efficiency']:.2%}")
print("Median VRAM efficiency:", f"{analysis_results['median_efficiency']:.2%}")

# Show report
print("\nReport:")
for rec in analysis_results["report"]:
    print("-", rec)
# Display efficiency patterns table
analysis_results["efficiency_patterns"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

inefficient_users = efficiency_analysis.find_inefficient_users_weighted_by_hours(
    min_jobs=5,  # Minimum number of jobs to consider a user
    efficiency_threshold=0.3  # Efficiency threshold for identifying inefficient users
)

# Display top inefficient users by job count
print("\nTop inefficient users by job count:")
display(inefficient_users.head(10))



# Plot top inefficient users by GPU hours
top_users = inefficient_users.head(10)

plt.figure(figsize=(7, 5))
sns.barplot(y=top_users["User"], x=top_users["Total_GPU_Hours"], orient="h")
plt.xlabel("GPU Hours")
plt.ylabel("User")
plt.title("Top 10 Inefficient Users by Weighted VRAM Efficiency Contribution")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

inefficient_pis = efficiency_analysis.find_inefficient_pis_weighted_by_hours(
    min_jobs=5,  # Minimum number of jobs to consider a user
    efficiency_threshold=0.3  # Efficiency threshold for identifying inefficient users
)

# Display top inefficient PI groups by job count
top_pis = inefficient_pis.head(10)

plt.figure(figsize=(7, 5))
sns.barplot(
    y=top_pis["Account"],
    x=top_pis["Total_GPU_Hours"],
    order=top_pis["Account"].tolist(),  # Only show present values
    orient="h"
)
plt.xlabel("GPU Hours")
plt.ylabel("PI Group")
plt.title("Top 10 Inefficient PI Groups by Weighted VRAM Efficiency Contribution")
plt.show()


In [ ]:
a100_h100_efficiency_analysis = efficiency_analysis.aggregate_metrics_by_gpu()
#convert to a matplotlib table
fig, ax = plt.subplots(figsize=(6, 2 + 0.5 * len(a100_h100_efficiency_analysis)))
ax.axis('off')

# Create the table
table = ax.table(
    cellText=a100_h100_efficiency_analysis.values,
    colLabels=a100_h100_efficiency_analysis.columns,
    rowLabels=a100_h100_efficiency_analysis.index,
    loc='center',
    cellLoc='center'
)


In [ ]:
# Check which columns are present in the jobs DataFrame
efficiency_analysis.additional_metrics()


In [ ]:
efficiency_jobs = efficiency_analysis.calculate_efficiency_metrics()
print("Columns in jobs_df for additional_metrics:")
print(efficiency_jobs.columns)
efficiency_analysis.additional_metrics()

# ...rest of your plotting code...

sns.set(style="whitegrid", font_scale=1.1)

fig, ax = plt.subplots(2, 1, figsize=(18, 14))
fig.subplots_adjust(hspace=0.4, wspace=0.3)

alloc_counts = efficiency_jobs['efficiency_category'].value_counts().sort_index()
sns.barplot(x=alloc_counts.index.astype(str), y=alloc_counts.values, ax=ax[0], palette="Blues_d")
ax[0].set_title('VRAM Allocation Efficiency Distribution')
ax[0].set_xlabel('Allocation Efficiency Category')
ax[0].set_ylabel('Number of Jobs')
ax[0].tick_params(axis='x', rotation=15)

req_counts = efficiency_jobs['duration_category'].value_counts().sort_index()
sns.barplot(x=req_counts.index.astype(str), y=req_counts.values, ax=ax[1], palette="Greens_d")
ax[1].set_title('Request Accuracy Distribution')
ax[1].set_xlabel('Request Accuracy Category')
ax[1].set_ylabel('Number of Jobs')
ax[1].tick_params(axis='x', rotation=15)

plt.show()






In [ ]:
import numpy as np
efficiency_analysis.calculate_efficiency_metrics()
# Only A100 and H100
cat_labels = [
    "Very Low (<10%)", "Low (10-30%)", "Medium (30-60%)", "High (60-100%)"
]
data_counts = {}
data_pcts = {}
gpu_types = (efficiency_analysis.efficiency_df['GPUType']
    .dropna()
    .explode()
    .astype(str)
    .str.strip()
    .str.lower()
    .unique()
)
for gpu in gpu_types:
    jobs = efficiency_analysis.efficiency_df[efficiency_analysis.efficiency_df['GPUType'].apply(
        lambda x, gpu=gpu: gpu in [str(g).strip().lower() for g in (x if isinstance(x, list | np.ndarray)
 else [x])]
    )]
    cat_counts = jobs["efficiency_category"].value_counts().reindex(cat_labels, fill_value=0)
    data_counts[gpu.upper()] = cat_counts.values
    total = cat_counts.sum()
    data_pcts[gpu.upper()] = [f"{(v/total*100):.1f}%" if total else "0.0%" for v in cat_counts.values]
# Build combined table: counts and percentages
table_data = []
for i, _cat in enumerate(cat_labels):
    row = []
    for gpu in gpu_types:
        row.append(f"{data_counts[gpu.upper()][i]}\n({data_pcts[gpu.upper()][i]})")
    table_data.append(row)
# Plot as PNG
fig, ax = plt.subplots(figsize=(8, 3 + 0.7 * len(cat_labels)))
ax.axis('off')
table = ax.table(
    cellText=table_data,
    rowLabels=cat_labels,
    colLabels=[g.upper() for g in gpu_types],
    loc='center',
    cellLoc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(2.0, 2.0)
ax.set_title(f"Efficiency Category Comparison ({80}GB)", fontweight='bold')
save_path = None
if save_path is None:
    save_path = f"eff_cat_{80}GB_comparison.png"
plt.savefig(save_path, bbox_inches='tight', dpi=100)
plt.close()
print(f"Saved efficiency category comparison table to {save_path}")
